# Client 동기화

In [3]:
import kfp
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://127.0.0.1:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)
list_pipelines = client.list_pipelines()

for i in range(list_pipelines.total_size):
    print(list_pipelines.pipelines[i].name)
print(response.url)
print(session_cookie)

[Demo] XGBoost - Iterative model training
[Demo] TFX - Taxi tip prediction model trainer
[Tutorial] Data passing in python components
[Tutorial] DSL - Control structures
example_pipeline
cheating-containerless
multiply-pipeline
mnist
iris-pipeline
http://127.0.0.1:8080/dex/auth/local/login?back=&state=nhze7mdzkwev5pq5ept5n5kju
MTY3MjYyMTY2OHxOd3dBTkZsQlVUVTNOVkkyV2xSQk4xTkhWVmRRVXpaSldrdEdXalZPTkVSSldqVlhOek15VlVVek5rUldORXRSVjAxQlVVdEVSbEU9fDo5az8iSmtfpSvxv8s278wbMfgibvNaumlBYtDiB_sE


In [6]:
client

# Experiment 정보 가져오기

In [41]:
experiment_info = client.get_experiment(experiment_name="mnist-pipeline",namespace=NAMESPACE)

In [42]:
experiment_id = experiment_info.id

# Experiment 삭제하기

In [43]:
client.delete_experiment(experiment_id=experiment_id)

{}

# Runs 삭제하기

In [44]:
for i in range(client.list_runs().total_size):
    run_id = client.list_runs().runs[i].id
    client.runs.delete_run(id=run_id)
    print(run_id)

6d768f1f-e224-4bd0-87d0-4d871bbc8c85


## Pipeline Upload

In [15]:
pipeline_package_path='/Users/gimgihun/kfp_practice/mnist/mnist.yaml'
pipeline_name='data'
client.upload_pipeline(pipeline_package_path=pipeline_package_path,pipeline_name=pipeline_name)

{'created_at': datetime.datetime(2023, 1, 2, 1, 55, 26, tzinfo=tzutc()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2023, 1, 2, 1, 55, 26, tzinfo=tzutc()),
                     'description': None,
                     'id': 'bb84b67c-0904-4ec3-bfa2-2cdfc1645a21',
                     'name': 'data',
                     'package_url': None,
                     'parameters': None,
                     'resource_references': [{'key': {'id': 'bb84b67c-0904-4ec3-bfa2-2cdfc1645a21',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': 'bb84b67c-0904-4ec3-bfa2-2cdfc1645a21',
 'name': 'data',
 'parameters': None,
 'resource_references': None,
 'url': None}

# Experiment 생성

In [16]:
experiment = client.create_experiment(name=pipeline_name,description=pipeline_name,namespace=NAMESPACE)

In [17]:
experiment_info = client.get_experiment(experiment_name=pipeline_name,namespace=NAMESPACE)
experiment_id = experiment_info.id
experiment_id


'08b489bb-d332-4a4d-a5b4-29b2c070c540'

## Run 생성

In [18]:
client.run_pipeline(experiment_id=experiment.id,job_name=pipeline_name,pipeline_package_path=pipeline_package_path)

ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'x-powered-by': 'Express', 'content-type': 'application/json', 'date': 'Mon, 02 Jan 2023 01:55:31 GMT', 'content-length': '775', 'x-envoy-upstream-service-time': '102', 'server': 'istio-envoy'})
HTTP response body: {"error":"Failed to create a new run.: InternalServerError: Failed to store run mnist-c5qbc to table: Error 1366: Incorrect string value: '\\xEB\\x8D\\xB0\\xEC\\x9D\\xB4...' for column 'WorkflowRuntimeManifest' at row 1","code":13,"message":"Failed to create a new run.: InternalServerError: Failed to store run mnist-c5qbc to table: Error 1366: Incorrect string value: '\\xEB\\x8D\\xB0\\xEC\\x9D\\xB4...' for column 'WorkflowRuntimeManifest' at row 1","details":[{"@type":"type.googleapis.com/api.Error","error_message":"Internal Server Error","error_details":"Failed to create a new run.: InternalServerError: Failed to store run mnist-c5qbc to table: Error 1366: Incorrect string value: '\\xEB